In [53]:
#1.Импорт Библиотек
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [54]:
#2.Создание датасетов на основании подгруженных данных
train_df = pd.read_csv('PD-data-train.csv', sep=';')
test_df = pd.read_csv('PD-data-test.csv', sep=';')
desc_df = pd.read_csv('PD-data-desc.csv', sep=';')

In [55]:
#5 верхних записей содержимого набора заголовков
desc_df.head()

,field,desc_eng,desc_rus
0,ar_revenue,Sales revenue (at the end of the last year),Выручка от реализации (на конец последнего года)
1,ar_total_expenses,Total expenses for the last year,Общие расходы за последний год
2,ar_sale_cost,Cost of sales (at the end of the last year),Себестоимость продаж (на конец последнего года)
3,ar_selling_expenses,Selling expenses (at the end of the last year),Коммерческие расходы (на конец последнего года)
4,ar_management_expenses,Administrative expenses (at the end of the las...,Управленческие расходы (на конец последнего года)


In [56]:
#5 верхних тренировочных записей
train_df.head()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,...,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,default_12m
0,7,52996000.0,48344000.0,48344000.0,0.0,0.0,4652000.0,-2608000.0,2044000.0,846000.0,...,3,3,3,[1-100],50000.0,2,0,0,0,0
1,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12,12,12,[1-100],10000.0,5,0,0,0,0
2,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28,28,28,[1-100],10000.0,2,0,0,0,0
3,22,32049000.0,29568000.0,29568000.0,0.0,0.0,2481000.0,-260000.0,2221000.0,302000.0,...,8,8,8,[1-100],300000.0,1,0,0,0,0
4,23,16458000.0,14935000.0,14935000.0,0.0,0.0,1523000.0,-274000.0,1249000.0,538000.0,...,27,27,27,[1-100],10000.0,2,0,0,0,0


In [57]:
#Размерность тренировочного датасета
train_df.shape

(32395, 37)

In [58]:
#5 верхних тестовых записей
test_df.head()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,...,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg
0,196,39103000.0,38913000.0,38913000.0,0.0,0.0,190000.0,141000.0,331000.0,284000.0,...,135,3,3,3,[1-100],100000.0,1,0,0,0
1,1196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,138,5,5,5,[1-100],100000.0,1,0,0,0
2,2813,54174000.0,50929000.0,50929000.0,0.0,0.0,3245000.0,-1278000.0,1967000.0,500000.0,...,138,20,20,20,[1-100],250000.0,2,0,0,0
3,4385,1904000.0,1679000.0,1679000.0,0.0,0.0,225000.0,-88000.0,137000.0,0.0,...,139,9,9,9,[1-100],10000.0,2,0,0,0
4,6479,225584000.0,210685000.0,210685000.0,0.0,0.0,14899000.0,-12715000.0,2184000.0,445000.0,...,139,3,3,3,(100-500],1700598.0,84,0,0,0


In [59]:
#Размерность тестового датасета
test_df.shape

(200, 36)

In [60]:
#Статистика типов данных в тренировочном датасете
train_df.dtypes.value_counts()

float64    25
int64      11
object      1
dtype: int64

In [61]:
#Средний дефолт-рейт в выборке
train_df['default_12m'].values.mean()

0.06454699799351751

In [62]:
#Количество уникальных записей в признаке "record_id" тренировчного датасета
train_df.record_id.nunique()

32395

In [63]:
#3.Построение бейзлайн модели
#Список исследуемых колонок-признаков
features = ['bus_age','ul_capital_sum', 'ul_founders_cnt']

In [64]:
#Создание нового датасета на основе тренировочного по выделенным ранее признакам
features_df = train_df.loc[:,features]

In [65]:
#Добавление колонки дефолт-рейта в новый датасет
features_df['default_12m'] = train_df['default_12m']

In [66]:
#Тестовые записи
features_df.head()

,bus_age,ul_capital_sum,ul_founders_cnt,default_12m
0,198,50000.0,2,0
1,256,10000.0,5,0
2,33,10000.0,2,0
3,185,300000.0,1,0
4,220,10000.0,2,0


In [67]:
#Получим карту корелляций по бейзлайн модели
features_df.corr()

,bus_age,ul_capital_sum,ul_founders_cnt,default_12m
bus_age,1.000000,0.066929,0.209451,-0.109389
ul_capital_sum,0.066929,1.000000,0.029165,-0.019452
ul_founders_cnt,0.209451,0.029165,1.000000,-0.023244
default_12m,-0.109389,-0.019452,-0.023244,1.000000


In [68]:
#Валидация результатов
#Заполняем записями соответсвующие значение выборок для обучения и тестирования (целевые переменные и побочные признаки)
X_train, X_test, y_train, y_test = train_test_split(features_df.drop('default_12m',axis=1),features_df['default_12m'],random_state=42, shuffle=True)

In [69]:
#Модель по методу градиентного бустинга
from sklearn.ensemble import GradientBoostingClassifier
grBoost = GradientBoostingClassifier(n_estimators=50, learning_rate=0.75, max_features=2, max_depth=2, random_state=241) #0.664283

In [70]:
#Кросс-валидация тренировочной модели с использованием Логистической регрессии и оценки результата по ROC-кривой (истинно-позитивные/истинно-негативные)
cv_baseline = cross_val_score(grBoost, X_train, y_train, cv=5, scoring='roc_auc').mean()
cv_baseline

0.6642833755002262

In [71]:
#Обучение модели на всех данных
grBoost.fit(features_df.drop('default_12m',axis=1), features_df['default_12m'])

GradientBoostingClassifier(learning_rate=0.75, max_depth=2, max_features=2,
                           n_estimators=50, random_state=241)

In [72]:
#Создаём файл с ответами
#Новый фрейм с записями и id
answ_df = test_df[['record_id']]

In [73]:
#Дублирование полученных значений id в новый столбец
answ_df['id'] = answ_df['record_id']

C:\Users\marin\AppData\Local\Temp/ipykernel_17040/3343517678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answ_df['id'] = answ_df['record_id']


In [74]:
#Удаляем первоначально скопированный из другого фрейма стобец и оставляем наш уникальный
answ_df.drop('record_id',axis=1, inplace=True)

C:\Python310\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [75]:
#Получаем среднее значение по целевой переменной из тренировочного датасета
cutoff = y_train.mean()

In [76]:
cutoff

0.06548403029305236

In [77]:
#Создаём новый столбец в фрейме ответов, содержащий в себе записи со значениями, превышающими по предикту cutoff из тестового датасета и конвертируем его тип bool в int
answ_df['predict'] = (grBoost.predict_proba(test_df.loc[:,features])[:,1]>cutoff).astype(int)

C:\Users\marin\AppData\Local\Temp/ipykernel_17040/3650119305.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answ_df['predict'] = (grBoost.predict_proba(test_df.loc[:,features])[:,1]>cutoff).astype(int)


In [78]:
#Результирующий фрейм 
answ_df

,id,predict
0,196,0
1,1196,0
2,2813,0
3,4385,0
4,6479,0
...,...,...
195,265577,1
196,266648,0
197,266849,1
198,267566,0


In [52]:
#Конвертируем финальный фрейм в csv-файл и сохраняем его для загрузки получившейся модели на сайт
answ_df.to_csv('PD-decision-submit-1.csv',index=False, sep=';')